# RAG (Retrieval-Augmented Generation)

## 概述
RAG（检索增强生成）是一种结合了信息检索与生成式模型的技术。它通过先检索相关文档，再利用生成模型（如大语言模型）对检索到的信息进行整合和生成，从而提升回答的准确性和丰富性。RAG广泛应用于问答系统、智能客服等场景，能够有效利用外部知识库，增强模型的知识覆盖面和实时性。

## 核心优势
- **知识实时性**：可以接入最新的外部知识库，避免模型知识过时
- **准确性提升**：基于检索到的真实信息生成答案，减少幻觉现象
- **可解释性**：可以追溯到答案的来源文档，增强可信度
- **成本效益**：无需重新训练大模型，通过检索机制扩展知识范围

## 适用场景
- 企业知识库问答系统
- 智能客服助手
- 文档检索与总结
- 个性化推荐系统

## 创建知识库

1. **嵌入向量**  
    使用嵌入模型（如BERT、SentenceTransformer等）将每个文本片段转换为高维向量，捕捉语义信息。

2. **写入向量库**  
    将生成的向量及其对应的原始文本片段存储到向量数据库（如FAISS、Milvus、Pinecone等），以便后续检索和调用。

### 创建向量库

In [1]:
from pymilvus import MilvusClient

milvus_client = MilvusClient("milvus_demo.db")

/Users/lipore/code/ainexushub/AI-101/.venv/lib/python3.13/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


### 创建数据集合

In [2]:
if milvus_client.has_collection(collection_name="demo_collection"):
    milvus_client.drop_collection(collection_name="demo_collection")
milvus_client.create_collection(
    collection_name="demo_collection",
    dimension=768,  # The vectors we will use in this demo has 768 dimensions
)

### 嵌入向量

In [3]:
from pymilvus import model

embedding_fn = model.DefaultEmbeddingFunction()

docs = [
    "人工智能作为一门学术学科创立于1956年。",
    "艾伦·图灵是第一位在人工智能领域进行实质性研究的人。",
    "图灵出生在伦敦的梅达维尔，成长于英格兰南部。",
]

vectors = embedding_fn.encode_documents(docs)
print("Dim:", embedding_fn.dim, vectors[0].shape)  # Dim: 768 (768,)

data = [
    {"id": i, "vector": vectors[i], "text": docs[i], "subject": "history"}
    for i in range(len(vectors))
]

print("Data has", len(data), "entities, each with fields: ", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


/Users/lipore/code/ainexushub/AI-101/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /GPTCache/paraphrase-albert-small-v2/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1142eecf0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 196af082-b4c3-4884-8381-c461bb8bd314)')' thrown while requesting HEAD https://huggingface.co/GPTCache/paraphrase-albert-small-v2/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


Dim: 768 (768,)
Data has 3 entities, each with fields:  dict_keys(['id', 'vector', 'text', 'subject'])
Vector dim: 768


### 保存数据到向量库

In [4]:
res = milvus_client.insert(collection_name="demo_collection", data=data)

print(res)

{'insert_count': 3, 'ids': [0, 1, 2]}


  
## 查询流程  
- 使用嵌入模型将问题转换为向量表示。  
- 在向量数据库中检索与问题向量最相似的文本片段。  
- 返回若干相关文本片段作为后续生成的上下文。



### 向量表示问题

In [5]:
query_vectors = embedding_fn.encode_queries(["谁是艾伦·图灵？"])
print("Query vector shape:", query_vectors[0].shape)  # Query vector shape: (768,)
print("Query vector:", query_vectors[0][:5])

Query vector shape: (768,)
Query vector: [-0.02033486  0.04349859 -0.05136999 -0.06811501 -0.02188697]


### 查询向量库

In [6]:
res = milvus_client.search(
    collection_name="demo_collection",  # target collection
    data=query_vectors,  # query vectors
    limit=2,  # number of returned entities
    output_fields=["text", "subject"],  # specifies fields to be returned
)

print(res)


data: [[{'id': 1, 'distance': 0.7951525449752808, 'entity': {'text': '艾伦·图灵是第一位在人工智能领域进行实质性研究的人。', 'subject': 'history'}}, {'id': 2, 'distance': 0.6408253908157349, 'entity': {'text': '图灵出生在伦敦的梅达维尔，成长于英格兰南部。', 'subject': 'history'}}]]


### 拼接上下文

In [7]:
import json

retrieved_lines_with_distances = [
    (r["entity"]["text"], r["distance"]) for r in res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

question = "谁是艾伦·图灵？"
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

[
    [
        "\u827e\u4f26\u00b7\u56fe\u7075\u662f\u7b2c\u4e00\u4f4d\u5728\u4eba\u5de5\u667a\u80fd\u9886\u57df\u8fdb\u884c\u5b9e\u8d28\u6027\u7814\u7a76\u7684\u4eba\u3002",
        0.7951525449752808
    ],
    [
        "\u56fe\u7075\u51fa\u751f\u5728\u4f26\u6566\u7684\u6885\u8fbe\u7ef4\u5c14\uff0c\u6210\u957f\u4e8e\u82f1\u683c\u5170\u5357\u90e8\u3002",
        0.6408253908157349
    ]
]


## 生成答案流程  
- 将检索到的文本片段与用户问题一起输入到生成式模型（如大语言模型）。  
- 生成式模型综合知识库内容和用户问题，理解上下文。  
- 输出针对用户问题的答案，答案中融合了知识库检索到的信息。  
- 将答案返回给用户。

### 构建LLM 客户端

In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import sys
sys.path.append(".")
import tools.message as message_tool
load_dotenv()
api_key = os.getenv("DEEPSEEK_API_KEY")
client = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

### 准备提示词

In [9]:
SYSTEM_PROMPT = """
你是一个AI助手。你能够根据提供的上下文段落片段找到问题的答案。
"""

USER_PROMPT = f"""
使用<context>标签中包含的以下信息，回答<question>标签中包含的问题。如果<context>标签中的信息无法回答问题，请说“对不起，我无法回答这个问题。”，而不是编造答案。
<context>
{context}
</context>
<question>
{question}
</question>
"""
messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ]

print(messages)

[{'role': 'system', 'content': '\n你是一个AI助手。你能够根据提供的上下文段落片段找到问题的答案。\n'}, {'role': 'user', 'content': '\n使用<context>标签中包含的以下信息，回答<question>标签中包含的问题。如果<context>标签中的信息无法回答问题，请说“对不起，我无法回答这个问题。”，而不是编造答案。\n<context>\n艾伦·图灵是第一位在人工智能领域进行实质性研究的人。\n图灵出生在伦敦的梅达维尔，成长于英格兰南部。\n</context>\n<question>\n谁是艾伦·图灵？\n</question>\n'}]


### 生成答案

In [10]:
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages
)
print(response.choices[0].message.content)

根据上下文信息，艾伦·图灵是第一位在人工智能领域进行实质性研究的人，他出生在伦敦的梅达维尔，成长于英格兰南部。
